In [1]:
import os
from collections import OrderedDict

import numpy as np
import matplotlib.pylab as plt
#%matplotlib inline
import time
from scipy import io as sio
import sklearn.metrics as sm
from sklearn.svm import LinearSVC

from keras.utils import np_utils
from tensorflow.python.client import device_lib

# TCN imports 
import tf_models, utils, metrics, datasets
from utils import imshow_
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
# import pickle
from keras.utils.vis_utils import plot_model
# import warnings
# warnings.filterwarnings('ignore')
# warnings.filterwarnings(action='once')
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Using TensorFlow backend.


## *** The lables for this has to be numbers ***
## Moreover the splits files has to be in a specific format the one that is generated in the "Preparing the data.ipynb" has to be corrected. The names must be in this format: 01 02 12 13 ...

### Make sure axes are correct (TxF not FxT for F=feat, T=time)

In [2]:
### Helper-function for plotting training history
def plot_training_history(dir_out,history,name_):
    # Make sure folder exists
    saveDir = os.path.join(dir_out,'plots')
    if not os.path.isdir(saveDir):
        os.makedirs(saveDir)
    # Get the classification accuracy and loss-value
    # for the training-set.
    acc = history.history['acc']
    loss = history.history['loss']

    # Get it for the validation-set (we only use the test-set).
    val_acc = history.history['val_acc']
    val_loss = history.history['val_loss']

    # Plot the accuracy and loss-values for the training-set.
    fig1=plt.figure()
    plt.plot(acc, linestyle='-', color='b', label='Training Acc.')
    plt.plot(loss/max(abs(np.array(loss))), 'o', color='b', label='Training Loss')
    plt.plot(val_acc, linestyle='-', color='r', label='Validation Acc.')
    plt.plot(val_loss/max(abs(np.array(val_loss))), 'o', color='r', label='Validation Loss')
    # Plot it for the test-set.
#     plt.plot(val_acc, linestyle='--', color='r', label='Test Acc.')
#     plt.plot(val_loss, 'o', color='r', label='Test Loss')

    # Plot title and legend.
    plt.title('Training Accuracy')
    plt.legend()

    # Ensure the plot shows correctly.
    plt.show()
    fig1.savefig(os.path.join(saveDir,'History_'+name_+'.png'))
    fig1.savefig(os.path.join(saveDir,'History_'+name_+'.eps'))

### 1- We choose the model properties and the dataset 

In [3]:
# ---------- Directories & User inputs --------------
# Location of data/features folder
base_dir = os.path.expanduser("../")
print(base_dir)
save_predictions = [False, True][1]
viz_predictions = [False, True][1]
viz_weights = [False, True][0]

# Set dataset and action label granularity (if applicable)
dataset = ["UW_IOM"][0]
print('Dataset: ', dataset)
granularity = ["eval", "mid"][0]
sensor_type = ["video", "sensors"][0]

# 1 - Set model and parameters
model_type = ["SVM", "LSTM",  "DilatedTCN", "ED-TCN"][3]
print('model_type:', model_type)
# causal or acausal? (If acausal use Bidirectional LSTM)
causal = [False, True][0]

# How many latent states/nodes per layer of network
# Only applicable to the TCNs. The ECCV and LSTM  model suses the first element from this list.
n_nodes = [64, 96]
# 2- Define the number of epochs
nb_epoch = 200 # Number of epochs
video_rate = 0 # sample rate. We don't have enough data to sample from
# 3- Defining the number of convolutional layer
conv = {'UW_IOM':20}[dataset]

# 4- Which features for the given dataset
features = ["FineTune45","PreTrained","PCNN"][1] 
print('Feature: ', features)
bg_class = 0 if dataset is not "JIGSAWS" else None


../
Dataset:  UW_IOM
model_type: ED-TCN
Feature:  PreTrained


In [ ]:

print(dataset+'_'+features+'_'+model_type)
if 1:
# for conv in [5, 10, 15, 20]:
    # Initialize dataset loader & metrics
    
    data = datasets.Dataset(dataset, base_dir)
    print(data)
    trial_metrics = metrics.ComputeMetrics(overlap=.1, bg_class=bg_class)
#     print(data.splits)
    # Load data for each split
    for split in data.splits:
        t = time.time()
        print('++++++++++++++++++++++++++++++++++++++++++++++++++++++' + split + '++++++++++++++++++++++++++++++++++++++++++++++++++++++')
        if sensor_type=="video":
            feature_type = "A" if model_type != "SVM" else "X"
        else:
            feature_type = "S"

        X_train, y_train, X_test, y_test = data.load_split(features, split=split, 
                                                            sample_rate=video_rate, 
                                                            feature_type=feature_type)

        if trial_metrics.n_classes is None:
            trial_metrics.set_classes(data.n_classes)

        n_classes = data.n_classes
        print('n_classes',n_classes)
        train_lengths = [x.shape[0] for x in X_train]
        test_lengths = [x.shape[0] for x in X_test]
        n_train = len(X_train)
        n_test = len(X_test)
        print('n_train',n_train ,'n_test',n_test)
        n_feat = data.n_features
        print("# Feat:", n_feat)

        # ------------------ Models ----------------------------
        if model_type == "SVM":
            
            svm = LinearSVC()
            t1 = time.time()
            svm.fit(np.vstack(X_train), np.hstack(y_train))
            ttrain = time.time()
            print("Training time")
            print(ttrain-t1)
            print("Training plus loading time")
            print(ttrain-t)
            AP_train = [svm.decision_function(x) for x in X_train]
            t2 = time.time()
            P_test = [svm.predict(x) for x in X_test]

            # AP_x contains the per-frame probabilities (or class) for each class
            AP_test = [svm.decision_function(x) for x in X_test]
            ttest = time.time()
            print("Test time")
            print(ttest - t2)
            param_str = "SVM"

        elif model_type in ["ED-TCN", "DilatedTCN", "LSTM"]:
            # Go from y_t = {1...C} to one-hot vector (e.g. y_t = [0, 0, 1, 0])
            Y_train = [np_utils.to_categorical(y, n_classes) for y in y_train]
            Y_test = [np_utils.to_categorical(y, n_classes) for y in y_test]


            # In order process batches simultaneously all data needs to be of the same length
            # So make all same length and mask out the ends of each.
            n_layers = len(n_nodes)
            max_len = max(np.max(train_lengths), np.max(test_lengths))
            max_len = int(np.ceil(max_len / (2**n_layers)))*2**n_layers
            print("Max length:", max_len)

            X_train_m, Y_train_, M_train = utils.mask_data(X_train, Y_train, max_len, mask_value=-1)
            X_test_m, Y_test_, M_test = utils.mask_data(X_test, Y_test, max_len, mask_value=-1)

            
            if model_type == "ED-TCN":
                model, param_str = tf_models.ED_TCN(n_nodes, conv, n_classes, n_feat, max_len, causal=causal, 
                                        activation='norm_relu', return_param_str=True)
                # plot the model
                plot_model(model, to_file=model_type+'_model.png', show_shapes=True)
                # model, param_str = tf_models.ED_TCN_atrous(n_nodes, conv, n_classes, n_feat, max_len, 
                                    # causal=causal, activation='norm_relu', return_param_str=True)                 
            elif model_type == "DilatedTCN":
#                 L = dilation_depth : number of layers per stack
#                 B = nb_stacks : number of stacks.
                L=3; B = 5
                model, param_str = tf_models.Dilated_TCN(n_feat, n_classes, n_nodes[0], L, B, max_len=max_len, 
                                        causal=causal, return_param_str=True)
                plot_model(model, to_file=model_type+'_model.png', show_shapes=True)
            elif model_type == "LSTM":
                model, param_str = tf_models.BidirLSTM(n_nodes[0], n_classes, n_feat, causal=causal, return_param_str=True)
                plot_model(model, to_file=model_type+'_model.png', show_shapes=True)
            # Run the model
            t1 = time.time()
            history = model.fit(X_train_m, Y_train_, epochs=nb_epoch, batch_size=8,
                        verbose=1,validation_data=[X_test_m,Y_test_] ,sample_weight=M_train[:,:,0]) 
            ttrain = time.time()
            print("Training time")
            print(ttrain-t1)
            print("Training plus loading model time")
            print(ttrain-t)
            AP_train = model.predict(X_train_m, verbose=0)
            AP_train = utils.unmask(AP_train, M_train)
            P_train = [p.argmax(1) for p in AP_train]
            
            t2 = time.time()
            AP_test = model.predict(X_test_m, verbose=0)
            AP_test = utils.unmask(AP_test, M_test)

            P_test = [p.argmax(1) for p in AP_test]
            ttest = time.time()
            print("Testing time")
            print(ttest-t2)

        # --------- Metrics ----------    
        trial_metrics.add_predictions(split, P_test, y_test)       
        trial_metrics.print_trials()
        

        # ----- Save predictions -----
        if save_predictions:
            dir_out = os.path.expanduser(base_dir+"/predictions/{}/{}/{}/".format(dataset,features,param_str))
            saveDir = os.path.join(dir_out,'plots')
            if not os.path.isdir(saveDir):
                os.makedirs(saveDir)
            # Make sure folder exists
            if not os.path.isdir(dir_out):
                os.makedirs(dir_out)

            out = {"P":P_test, "Y":y_test, "S":AP_test}
            sio.savemat( dir_out+"/{}.mat".format(split), out)      

        # ---- Viz predictions -----
        if viz_predictions:
            max_classes = data.n_classes - 1
            # # Output all truth/prediction pairs
            fig = plt.figure(split, figsize=(20,10))
            P_test_ = np.array(P_test)#/float(n_classes-1)
#             print(P_test); print(y_test)
            y_test_ = np.array(y_test)#/float(n_classes-1)
            for i in range(len(y_test)):
                P_tmp = np.vstack([y_test_[i], P_test_[i]])
                plt.subplot(n_test,1,i+1)
                imshow_(P_tmp, vmin=0, vmax=1)
                plt.xticks([])
                plt.yticks([])
#                 plt.legend()
                acc = np.mean(y_test[i]==P_test[i])*100
                plt.ylabel("Acc: {:.01f}%".format(acc))
#                 plt.title("Acc: {:.03}%".format(100*np.mean(P_test[i]==y_test[i])))
            fig.savefig(os.path.join(saveDir,'prediction_'+dataset+'_'+features+'_'+model_type+'_'+split+'.png'))
            fig.savefig(os.path.join(saveDir,'prediction_'+dataset+'_'+features+'_'+model_type+'_'+split+'.eps'), format="eps")
#             pickle.dump(fig,file('prediction.pickle','w'))
        # ---- Viz weights -----
        if viz_weights and model_type is "TCN":
            # Output weights at the first layer
            plt.figure(2, figsize=(15,15))
            ws = model.get_weights()[0]
            for i in range(min(36, len(ws.T))):
                plt.subplot(6,6,i+1)
                # imshow_(model.get_weights()[0][i][:,:,0]+model.get_weights()[1][i])
                imshow_(np.squeeze(ws[:,:,:,i]).T)
                # Output weights at the first layer

            for l in range(2*n_layers):
                plt.figure(l+1, figsize=(15,15))
                ws = model.get_weights()[l*2]
                for i in range(min(36, len(ws.T))):
                    plt.subplot(6,6,i+1)
                    # imshow_(model.get_weights()[0][i][:,:,0]+model.get_weights()[1][i])
                    imshow_(np.squeeze(ws[:,:,:,i]).T)
                    # Output weights at the first layer
        #plot_training_history(dir_out,history,features+'_'+model_type+'_'+split)
        elapsed = time.time() - t
        print('Time elapsed: ', elapsed)
    print()
    trial_metrics.print_scores()
    trial_metrics.print_trials()
    print()
#     plot_training_history(history,model_type+'_'+split)
##### Saving the Model #####
# serialize model to YAML
model_yaml = model.to_yaml()
with open(dir_out+dataset+'_'+features+'_'+model_type+".yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)
# serialize weights to HDF5
model.save_weights(dir_out+dataset+'_'+features+'_'+model_type+".h5")
print("Saved model to disk")
fig.savefig(os.path.join(dir_out+'/plots/','prediction_'+dataset+'_'+features+'_'+model_type+'_'+split+'.png'))

elapsed = time.time() - t
print('Time elapsed: ', elapsed)

### Save the model summary

In [ ]:
print('Model summary:')
print(model.summary())

######################################################################################################################